## Q-learning en una màquina expenedora intel·ligent

#### Ada López del Castillo Avilés
#### NIU: 1605347

### Exercici 1: Afegir un estat addicional (L: estoc baix)


In [1]:
import numpy as np
import random

In [2]:
# Paràmetres
alpha = 0.5
gamma = 0.9
epsilon = 0.3
n_iter = 100

# Estats i accions 
states = ["C", "NC", "L"] # C: Clients, NC: No clients, L: Low stock
actions = [1, 2, 3]  # 1: Promo forta, 2: Promo suau, 3: No fer res

#Inicialització de la taula amb valors 0
Q = {s: {a: 0 for a in actions} for s in states}

#Funció recompensa
#Afegim les recompenses del Low Stock, imposant que només sigui positiva l'acció 3
def get_reward(state, action):
    if state == "C":
        return {1: 6, 2: 3, 3: 0}[action]
    elif state == "NC":
        return {1: -3, 2: -1, 3: 1}[action]
    elif state == "L":
        return {1: -5, 2: -3, 3: 2}[action]
#Fem un comptador per controlar el número de promocions que fa
comptador = 0

#Funció transició amb probabilitats
def next_state(state, action, comptador):

    #Afegim que si fem acció 1 o 2 (és a dir, qualsevol promo), es suma 1 al comptador.
    if action in [1,2]:
        comptador += 1
    
    #Imposem que si s'arriba a més de 5 promocions, passem directament al estat L, ja que per excès de promocions ens hem quedats sense productes.
    if comptador > 5:
        return "L", comptador 
        
    if state == "C":
        return (np.random.choice(["NC", "C"], p=[0.7, 0.3]), comptador) if action in [1, 2] else (np.random.choice(["C", "NC"], p=[0.4, 0.6]), comptador)
    
    elif state == "NC":
        return np.random.choice(["C", "NC"], p=[0.5, 0.5]), comptador

    #Afegim llavors que si estem en l'estat L, per excès de promocions, el comptador torna a 0 perquè hi ha hagut reposició (ho suposem)
    #A més, li diem que només passi a estar Low Stock o NoClients, amb prob 0.4 i 0.6 respectivament. 
    elif state == "L":
        comptador = 0 
        return np.random.choice(["L", "NC"], p=[0.4, 0.6]), comptador        

#Notem que hem hagut d'afegir 'comptador' a cadascun dels returns, per guardar la informació i així saber quan arribem al màxim de promos.

#Funció acció aleatoria amb e-greedy
def choose_action(state):
    if random.random() < epsilon:
        return random.choice(actions)
    qvals = Q[state]
    max_val = max(qvals.values())
    best_actions = [a for a in actions if qvals[a] == max_val]
    return random.choice(best_actions)

state = "C"

#Bucle principal de Q-learning
for i in range(n_iter):
    action = choose_action(state)
    reward = get_reward(state, action)
    next_s, comptador = next_state(state, action, comptador)
    Q[state][action] += alpha * (reward + gamma * max(Q[next_s].values()) - Q[state][action])
    state = next_s
#S'ha hagut de posar el comptador per tal d'evitar errors

# Resultats
print("Q-table:")
for s in Q:
    print(f"{s}: {Q[s]}") 

Q-table:
C: {1: 9.409691531250001, 2: 5.000945543671875, 3: 5.459486189062501}
NC: {1: -1.5, 2: 0, 3: 7.779878985937501}
L: {1: 5.938257882958073, 2: 13.841464220608106, 3: 19.38872708173352}


### Exercici 2: Accions estocàstiques

In [3]:
#En aquest apartat només calia canviar la funció get_reward per fer les accions estocàstiques, la resta del codi es manté

In [4]:
alpha = 0.5
gamma = 0.9
epsilon = 0.3
n_iter = 100

states = ["C", "NC", "L"] 
actions = [1, 2, 3]  

Q = {s: {a: 0 for a in actions} for s in states}

#Observem que treiem el return de les recompenses per poder aplicar estoicisitat,
#i les guardem amb el nom reward, per a més endavant depenent de la acció que sigui, que passi una cosa o una altra.
def get_reward(state, action):
    if state == "C":
        reward = {1: 6, 2: 3, 3: 0}[action]
    elif state == "NC":
        reward = {1: -3, 2: -1, 3: 1}[action]
    elif state == "L":
        reward = {1: -5, 2: -3, 3: 2}[action]
    
    #Acció 1 falla un 10% i no ven res, per tant pèrdua amb recompensa -1
    if action == 1:
        return np.random.choice([reward, -1], p=[0.9, 0.1])

    #Acció 2 redueix la recompensa en un 20% dels casos, la qual he decidit que sigui una reducció de 2.
    if action == 2:
        return np.random.choice([reward, reward - 2], p=[0.8, 0.2])
    
    #Acció 3 en un 5% dels casos falla en no mantenir clients, amb una petita penalització de -1.
    if action == 3:
        return np.random.choice([reward, -1], p=[0.95, 0.05])

comptador = 0

def next_state(state, action, comptador):

    if action in [1,2]:
        comptador += 1
    
    if comptador > 5:
        return "L", comptador 
        
    if state == "C":
        return (np.random.choice(["NC", "C"], p=[0.7, 0.3]), comptador) if action in [1, 2] else (np.random.choice(["C", "NC"], p=[0.4, 0.6]), comptador)
    
    elif state == "NC":
        return np.random.choice(["C", "NC"], p=[0.5, 0.5]), comptador

    elif state == "L":
        comptador = 0 
        return np.random.choice(["L", "NC"], p=[0.4, 0.6]), comptador 

def choose_action(state):
    if random.random() < epsilon:
        return random.choice(actions)
    qvals = Q[state]
    max_val = max(qvals.values())
    best_actions = [a for a in actions if qvals[a] == max_val]
    return random.choice(best_actions)

state = "C"

for i in range(n_iter):
    action = choose_action(state)
    reward = get_reward(state, action)
    next_s, comptador = next_state(state, action, comptador)
    Q[state][action] += alpha * (reward + gamma * max(Q[next_s].values()) - Q[state][action])
    state = next_s

print("Q-table:")
for s in Q:
    print(f"{s}: {Q[s]}") 
    

Q-table:
C: {1: np.float64(3.0), 2: np.float64(1.9543750000000002), 3: np.float64(0.8794687500000001)}
NC: {1: np.float64(-1.5), 2: np.float64(-0.5), 3: np.float64(2.2410312500000003)}
L: {1: np.float64(10.063488094382052), 2: np.float64(13.836669907262097), 3: np.float64(19.22077465647504)}


### Exercici 3: Impacte dels paràmetres d'aprenentatge


In [5]:
#Fem una funció que cridi els valors dels paràmetres per tal de no possar-ho manualment

In [8]:
states = ["C", "NC", "L"]
actions = [1, 2, 3] 

def get_reward(state, action):
    if state == "C":
        reward = {1: 6, 2: 3, 3: 0}[action]
    elif state == "NC":
        reward = {1: -3, 2: -1, 3: 1}[action]
    elif state == "L":
        reward = {1: -5, 2: -3, 3: 2}[action]
    
    if action == 1:
        return np.random.choice([reward, -1], p=[0.9, 0.1])
    if action == 2:
        return np.random.choice([reward, reward - 2], p=[0.8, 0.2])
    if action == 3:
        return np.random.choice([reward, -1], p=[0.95, 0.05])

def next_state(state, action, comptador):

    if action in [1,2]:
        comptador += 1
    
    if comptador > 5:
        return "L", comptador 
        
    if state == "C":
        return (np.random.choice(["NC", "C"], p=[0.7, 0.3]), comptador) if action in [1, 2] else (np.random.choice(["C", "NC"], p=[0.4, 0.6]), comptador)
    
    elif state == "NC":
        return np.random.choice(["C", "NC"], p=[0.5, 0.5]), comptador

    elif state == "L":
        comptador = 0 
        return np.random.choice(["L", "NC"], p=[0.4, 0.6]), comptador        

#Fem una funció que executi tot l'algoritme
def Q():
    #Provarem totes les combinacions de alpha, gamma i epsilon possibles entre 0.1 i 0.9 amb un increment de 0.1, a través de bucles que cridin a aquests valors.
    valor= np.arange(0.1, 1.0, 0.1) 
    
    for alpha in valor:
        for gamma in valor:
            for epsilon in valor:
                #Posem el comptador, l'estat pel que comencem i el nombre de iteracions de cada execució dins de la funció
                #La inicialització també a dins per tal de que siguin independents les execucions i comencin en 0
                Q = {s: {a: 0 for a in actions} for s in states}
                comptador = 0
                state = "C"
                n_iter = 100
                
                def choose_action(state):
                    if random.random() < epsilon:
                        return random.choice(actions)
                    qvals = Q[state]
                    max_val = max(qvals.values())
                    best_actions = [a for a in actions if qvals[a] == max_val]
                    return random.choice(best_actions)
                
                for i in range(n_iter):
                    action = choose_action(state)
                    reward = get_reward(state, action)
                    next_s, comptador = next_state(state, action, comptador)
                    Q[state][action] += alpha * (reward + gamma * max(Q[next_s].values()) - Q[state][action])
                    state = next_s

                #Per a que mostri els resultats de cada combinació, indicant quins són els paràmetres
                print(f"\n alpha={alpha}, gamma={gamma}, epsilon={epsilon}")
                for s in Q:
                    print(f"{s}: {Q[s]}")
#Si volem veure les 729 combinacions només s'ha d'escriure Q() i executar
#Si volem veure de manera més general, canviem l'increment a 0.3 per tal de veure les 27 combinacions.